# Vector Addition in SYCL Using Buffers/Accessors

In the example in the cell below, the code performs vector addition using the buffer/accessor model. Vectors `A` and `B` are added element-wise, and the results are stored in vector `C`. Unlike the Unified Shared Memory (USM) model, where memory is directly shared between the host and device, the buffer/accessor model explicitly handles data movement and synchronization:

* Buffers (`bufA`, `bufB`, `bufC`) are created to manage data between host and device.
* Accessors (`accA`, `accB`, `accC`) define how the kernel accesses the data within those buffers, ensuring synchronization and correct data handling.
* SYCL automatically manages the transfer of data between the host and the device based on the access mode specified by the accessors.

This approach is beneficial when you want SYCL to handle the complexities of data synchronization, whereas USM would require you to manually manage memory sharing and synchronization.

In [ ]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>

int main(int argc, char **argv) {
  const int N = 2;

  std::vector<int> vectorA(N, 1);  // Vector A filled with 1s
  std::vector<int> vectorB(N, 2);  // Vector B filled with 2s
  std::vector<int> vectorC;        // Vector C initialized with no specific values
  vectorC.resize(vectorA.size());

  auto myQueue = sycl::queue{sycl::gpu_selector_v};
  std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << "\n";

  {
      auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};
      auto bufB = sycl::buffer{vectorB.data(), sycl::range{N}};
      auto bufC = sycl::buffer{vectorC.data(), sycl::range{N}};
    
      myQueue.submit([&](sycl::handler &cgh) {

      sycl::accessor accA { bufA, cgh, sycl::read_only};
      sycl::accessor accB { bufB, cgh, sycl::read_only };
      sycl::accessor accC { bufC, cgh, sycl::write_only, sycl::no_init };

      cgh.parallel_for(N, [=](auto i) { accC[i] = accA[i] + accB[i]; });

    });
  }

  // Print out the result of vector add
  int indices[]{0, 1, 2, (static_cast<int>(vectorA.size()) - 1)};
  constexpr size_t indices_size = sizeof(indices) / sizeof(int);
  for (int i = 0; i < indices_size; i++) {
    int j = indices[i];
    if (i == indices_size - 1) std::cout << "...\n";
    std::cout << "[" << j << "]: " << vectorA[j] << " + " << vectorB[j] << " = "
              << vectorC[j] << "\n";

  }

  return 0;
}


The cell below creates a script that sets up the environment, compiles the SYCL program using Intel’s DPC++ compiler, and runs the executable if the compilation is successful.

In [ ]:
%%writefile ./run-sycl.sh

#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

This command makes the script executable and then runs it, executing the compiled SYCL program to perform the vector addition.

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh